## SQL com Python + Pandas + SQLalchemy 

In [ ]:
# CASO NÃO TENHA AS DEPENDÊNCIAS INSTALADOS 
! pip install pandas sqlalchemy openpyxl 

In [2]:
# IMPORTAÇÃO DOS PACOTES NECESSÁRIOS 
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql

### Usaremos o SQLite3 como nosso banco de dados relacional, pois é muito leve e fácil de usar.  Embora a biblioteca SQLAlchemy possa se conectar a uma variedade de fontes relacionais, incluindo MySql, Oracle e Postgresql e Mssql.  Primeiro criamos um mecanismo de banco de dados e, em seguida, nos conectamos a ele usando a função to_sql da biblioteca SQLAlchemy. No exemplo a seguir, criamos a tabela relacional usando a função to_sql de um dataframe já criado pela leitura de um arquivo xlsx.  Em seguida, usamos a função read_sql_query do pandas para executar e capturar os resultados de várias consultas SQL.

## Criando e consultando o banco de dados com SQLite (DQL)

In [3]:
#Carrega o arquivo Excel com pandas 
data = pd.read_excel('Planilha_to_SQL.xlsx', engine='openpyxl')

# Criação do motor de banco de dados com SQlite rodando em memória 
engine = create_engine('sqlite:///:memory:')

# Armazena o dataframe como uma tabela no disco
data.to_sql('data_table', engine)

# Primeira Query na tabela relacional 
# Selecionei a tabela inteira , carregada pelo motor "engine".
res1 = pd.read_sql_query('SELECT * FROM data_table', engine)
print('Resultado 1')
print(res1)
print('')

# Segunda Query
# Agupei por departamento e somei os salários distribuidos por departamento da tabela "data_table"
res2 = pd.read_sql_query('SELECT dept,sum(salary) FROM data_table group by dept', engine)
print('Resultado 2')
print(res2)


Resultado 1
   index  id    name  salary                  start_date        dept
0      0   1   Rick      623  2012-01-01 00:00:00.000000          IT
1      1   2     Dan     515  2013-09-23 00:00:00.000000  Operations
2      2   3   Tusar     611  2014-11-15 00:00:00.000000          IT
3      3   4    Ryan     729  2014-05-11 00:00:00.000000          HR
4      4   5    Gary     843  2015-03-27 00:00:00.000000     Finance
5      5   6   Rasmi     578  2013-05-21 00:00:00.000000          IT
6      6   7  Pranab     632  2013-07-30 00:00:00.000000  Operations
7      7   8    Guru     722  2014-06-17 00:00:00.000000     Finance

Resultado 2
         dept  sum(salary)
0     Finance         1565
1          HR          729
2          IT         1812
3  Operations         1147


## Inserindo dados na tabela  ( DML ) e ( DQL )

### Também podemos inserir dados em tabelas relacionais usando a função sql.execute disponível no pandas.  No código a seguir o arquivo xlsx anterior é  usado como conjunto de dados de entrada, armazenamos em uma tabela relacional e então inserimos outro registro usando sql.execute.

In [4]:
data = pd.read_excel('Planilha_to_SQL.xlsx',engine='openpyxl')
engine = create_engine('sqlite:///:memory:')

# Armazene os dados em uma tabela relacional
data.to_sql('data_table', engine)

# Inserir a nova linha na tabela / preencher com os valores  
sql.execute('INSERT INTO data_table VALUES(?,?,?,?,?,?)', 
            engine, params=[('id',9,'Ruby',711.20,'2015-03-27','IT')])

# Leitura da tabela / Query  com todos atributos mais a nova linha 
res = pd.read_sql_query('SELECT ID,Dept,Name,Salary,start_date FROM data_table',
                        engine)

print(res)

   id        dept    name  salary                  start_date
0   1          IT   Rick    623.0  2012-01-01 00:00:00.000000
1   2  Operations     Dan   515.0  2013-09-23 00:00:00.000000
2   3          IT   Tusar   611.0  2014-11-15 00:00:00.000000
3   4          HR    Ryan   729.0  2014-05-11 00:00:00.000000
4   5     Finance    Gary   843.0  2015-03-27 00:00:00.000000
5   6          IT   Rasmi   578.0  2013-05-21 00:00:00.000000
6   7  Operations  Pranab   632.0  2013-07-30 00:00:00.000000
7   8     Finance    Guru   722.0  2014-06-17 00:00:00.000000
8   9          IT    Ruby   711.2                  2015-03-27


## Excluindo dados na tabela ( DML ) e ( DQL )

### Também podemos deletar dados em tabelas relacionais usando a função sql.execute disponível no pandas.  O código a seguir exclui uma linha com base na condição de entrada fornecida.

In [5]:
data = pd.read_excel('Planilha_to_SQL.xlsx',engine='openpyxl')
engine = create_engine('sqlite:///:memory:')

data.to_sql('data_table', engine)
# Excluí linha na tabela onde o nome é igual à Gary 
sql.execute('Delete from data_table where name = (?) ', engine, params=[('Gary')])
# Leitura da tabela com atributos completos, atualizada
res = pd.read_sql_query('SELECT ID,Dept,Name,Salary,start_date FROM data_table',engine)
# Exibe 
print(res)

   id        dept    name  salary                  start_date
0   1          IT   Rick      623  2012-01-01 00:00:00.000000
1   2  Operations     Dan     515  2013-09-23 00:00:00.000000
2   3          IT   Tusar     611  2014-11-15 00:00:00.000000
3   4          HR    Ryan     729  2014-05-11 00:00:00.000000
4   6          IT   Rasmi     578  2013-05-21 00:00:00.000000
5   7  Operations  Pranab     632  2013-07-30 00:00:00.000000
6   8     Finance    Guru     722  2014-06-17 00:00:00.000000
